# Import packages

In [1]:
import os
import time 
import numpy as np
import pandas as pd
import geopandas as gp

In [2]:
from sklearn.linear_model import LinearRegression

In [3]:
import xarray as xr
import rioxarray
from rasterio.enums import Resampling

In [4]:
# Time the script
start_time = time.perf_counter ()

# Set up directories and file paths

In [5]:
path_data_in = '../2 Raw Data'
path_data_out = '../3 Output Data'
path_map = '../4 Mapping'

## Input data

In [6]:
path_pop_count = os.path.join(path_data_in, 
                              'Population', 
                              'Count',
                              'zaf_ppp_2015_1km_Aggregated.tif')  


dir_pop_agesex= os.path.join(path_data_in, 
                             'Population', 
                             'AgeSex')
#file_healthsiteIO_shp=os.path.join(path_data_in, 
                                 #  'Facilities',
                                  # 'Uzbekistan_HealthsitesIO_SHP',
                                  # "Uzbekistan-node.shp")
#file_healthsiteIO_csv=os.path.join(path_data_out,
                                #   'Uzbekistan_healthsites_io.csv')
path_gdp=os.path.join(path_data_in,
                      'GDP_HDI',
                      'GDP_PPP_30arcsec_v3.nc')


path_hotdays =os.path.join(path_map,
                      'anomaly-hd35-annual-mean_cmip6_annual_all-regridded-bct-ssp119-climatology_median_2020-2039.nc')  

     
path_heatindex =os.path.join(path_map,
                     'anomaly-hi35-annual-mean_cmip6_annual_all-regridded-bct-ssp119-climatology_median_2020-2039.nc')                      


path_nl = os.path.join(path_data_in,
                       'Nighttime_Lights',
                       'zaf_viirs_100m_2015.tiff')
#no birth data for Albania
#path_birth=os.path.join(path_data_in,
 #                       'Birth',
  #                      'Uzbekistan_1km_Births',
   #                     'uzb_births_pp_v2_2015.tif')
# path_death = os.path.join(path_data_in,
#                           'Death',
#                           'Albania_death_data_by_age_and_sex_2019.csv') 

## Output data

In [7]:
#path_healthfacility_public = os.path.join(path_data_out,"public_health_facilities.csv")
#path_healthfacility_private = os.path.join(path_data_out,"private_health_facilities.csv")
path_agesex = os.path.join(path_data_out,"age_sex_distribution_1km_2019.csv")
#bank_csv_path = os.path.join(data_in_path,"Egypt_Banks_and_ Atms.csv")

# User-defined functions

In [8]:
def find_grid_index(x,y,longs,lats,x_min=None,x_max=None,y_min=None,y_max=None):
    """
    DESCRIPTION: This function helps to find the index of the longitudes and 
    latitudes in the given longitude/latitude lists that is closest to the 
    given point (x,y) 
    
    INPUT -
    x: latitude of the given point
    y: longtidue of the given point
    longs: reference list of longitudes
    lats: reference list of latitudes
    x_min, x_max: min/max boundaries of longitude
    y_min, y_max: min/max boundaries of latitude
    
    OUTPUT -
    Index of the closest longitude/latitude to point (x,y)
    """
    if x_min is None and x_max is None:
        x_l = longs[0]- (longs[1]-longs[0])
        x_r = longs[-1] + (longs[-1]-longs[-2])
        x_min = min(x_l, x_r)
        x_max = max(x_l, x_r)
    if x < x_min or x > x_max:
        return -1,-1
    
    if y_min is None and y_max is None:
        y_l = lats[0]- (lats[1]-lats[0])
        y_r = lats[-1] + (lats[-1]-lats[-2])
        y_min = min(y_l, y_r)
        y_max = max(y_l, y_r)
    if y < y_min or y > y_max:
        return -1,-1
    
    ncol = np.argmin(abs(longs-x)) 
    nrow = np.argmin(abs(lats-y))
    if nrow<0 or nrow>=(len(lats)-1) or ncol<0 or ncol>=(len(longs)-1):
        return -1, -1
    else:
        return nrow,ncol

In [9]:
def points_in_grid_new(points_df,ref_data,value_colname=None):
    """
    DESCRIPTION: This function finds how many points (and how many non nan points and their sum  
    if a value_colname is provided) lying in the grids centered at the coordinates of the reference data
    
    INPUT -
    points_df: pandas dataframe that needs to be count 
    ref_data: xarray DataArray with reference coordinates
    value_colname: column name for the values in the dataframe
    
    OUTPUT -
    count: How many points are in each grid of the reference coordinate
    non_nan_count: How many non nan points are in each grid of the reference coordinate
    value_sum: The sum of the values of the non nan points within each grid of the reference coordinate
    """
    long_list = ref_data.coords['x'].values
    lat_list = ref_data.coords['y'].values
    long_list = np.append(long_list, long_list[-1]+ref_data.rio.resolution()[0])
    lat_list = np.append(lat_list, lat_list[-1]+ref_data.rio.resolution()[1])
    count = np.zeros(ref_data.values.shape)
    non_nan_count = np.zeros(ref_data.values.shape)
    if value_colname is not None:
        value_sum = np.empty(ref_data.values.shape)
        value_sum[:] = np.nan
        values = points_df[value_colname]
    else:
        value_sum = None
    grid_index = points_df.apply(lambda row: find_grid_index(row['Long'],row['Lat'],long_list,lat_list), axis=1)
    for j, x in enumerate(grid_index):
        if x[0]!=-1 and x[1]!=-1:
            count[x[0],x[1]] += 1
            if value_colname is not None and ~np.isnan(values[j]):
                non_nan_count[x[0],x[1]] = non_nan_count[x[0],x[1]]+1 
                value_sum[x[0],x[1]] = np.nansum([value_sum[x[0],x[1]],values[j]])
    return count, non_nan_count, value_sum

In [10]:
def spatial_aggregate(count_in,value_sum_in,block_size,agg_mode='sum', non_nan_count_in=None):
    """
    DESCRIPTION: This function apply square sum aggregation with stride 1 for 2D numpy array
    with the same padding
    
    INPUT -
    count_in: 2D numpy array for the count at each grid
    value_sum_in: 2D numpy array for the sum of values at each grid
    block_size: the size of the square sum aggregation
    agg_mode: aggregation operation mode "sum" or "mean"
    non_nan_count_in: 2D array for the non nan count at each grid, only needed for "mean" mode
    
    
    OUTPUT - 
    a new 2D numpy array with square sum aggreation in the same size
    """
    if block_size%2 != 1:
        raise Exception('block_size must be an odd integer!')
    half = block_size//2

    new_count_in = np.zeros(count_in.shape)
    count_in = np.pad(count_in,half)

    return_value = np.zeros(value_sum_in.shape)
    value_sum_in = np.pad(value_sum_in,half)
    
    if agg_mode == 'mean':
        new_nonnan_count_in = np.zeros(non_nan_count_in.shape)
        non_nan_count_in = np.pad(non_nan_count_in,half)
    elif agg_mode == 'sum':
        new_nonnan_count_in = None
    else:
        raise Exception('Error: The function only supports agg_mode for sum or mean.')
    for nrow in range(half,count_in.shape[0]-half):
        for ncol in range(half,count_in.shape[1]-half):
            new_count_in[nrow-half,ncol-half] = np.sum(count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            if agg_mode == 'mean':
                new_nonnan_count_in[nrow-half,ncol-half] = np.sum(non_nan_count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            return_value[nrow-half,ncol-half] = np.nansum(value_sum_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
    if agg_mode == 'mean':
        new_nonnan_count_in = np.where(new_nonnan_count_in == 0, np.nan, new_nonnan_count_in)
        return_value = return_value/new_nonnan_count_in
        return new_count_in, return_value, new_nonnan_count_in
    else:
        return new_count_in, return_value, new_nonnan_count_in

In [11]:
def layer_aggregate_helper(count_in,value_sum_in,block_size,non_nan_count_in):
    """
    DESCRIPTION: This function apply square sum aggregation with stride 1 for 2D numpy array
    with the same padding
    
    INPUT -
    count_in: 2D numpy array for the count at each grid
    value_sum_in: 2D numpy array for the sum of values at each grid
    block_size: the size of the square sum aggregation
    non_nan_count_in: 2D array for the non nan count at each grid
    
    
    OUTPUT - 
    a new 2D numpy array with square sum aggreation in the same size
    """
    if block_size%2 != 1:
        raise Exception('block_size must be an odd integer!')
    half = block_size//2

    new_count_in = np.zeros(count_in.shape)
    count_in = np.pad(count_in,half)

    return_value = np.zeros(value_sum_in.shape)
    value_sum_in = np.pad(value_sum_in,half)
    
    new_nonnan_count_in = np.zeros(non_nan_count_in.shape)
    non_nan_count_in = np.pad(non_nan_count_in,half)
  
    for nrow in range(half,count_in.shape[0]-half):
        for ncol in range(half,count_in.shape[1]-half):
            new_count_in[nrow-half,ncol-half] = np.sum(count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            new_nonnan_count_in[nrow-half,ncol-half] = np.sum(non_nan_count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            return_value[nrow-half,ncol-half] = np.nansum(value_sum_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
    return_value = np.where(new_nonnan_count_in == 0, np.nan, return_value)
    return new_count_in, return_value, new_nonnan_count_in

In [12]:
def layer_aggregate(value1,count1,value2,count2,mode = 'sum'):
    value1 = np.array(value1)
    value2 = np.array(value2)
    count1 = np.array(count1)
    count2 = np.array(count2)
    value2[np.where(count1==count2)]=value1[np.where(count1==count2)]
    value1 = np.where(np.isnan(value1), 0, value1)
    value2 = np.where(np.isnan(value2), 0, value2)
    value_diff = value2-value1
    count_diff = count2-count1
    if mode == 'sum':
        return_value = value_diff
        return_value = np.where(count_diff==0,np.nan,return_value)
    elif mode == 'mean':
        count_diff = np.where(count_diff == 0, np.nan, count_diff)
        return_value = value_diff/count_diff
    else:
        raise Exception('Only support sum or mean for mode')
    return return_value

In [13]:
def square_helper(arr, y, x, mode):
    """
    DESCRIPTION: This is a helper funtion to do mean or sum in square for 2D numpy array
    
    INPUT - 
    arr: 2D numpy array to be averaged
    (y,x): output dimension
    mode: sum or mean, NAs are ignored 

    OUTPUT - 
    Averaged 2D array
    """
    yy, xx = arr.shape
    if mode == "sum":
        vals = np.nansum(arr.reshape(y, yy//y, x, xx//x),(1,3))
    elif mode == "mean":
        vals = np.nanmean(arr.reshape(y, yy//y, x, xx//x),(1,3))
    else:
        raise Exception("Mode is not suppported, please input `sum' or `mean'")
    return vals

In [14]:
def xarray_square_aggregate(df_raw,stride=10,var_name="band_data",mode="mean"):
    """
    DESCRITPION: This is a function to do square aggregation (mean or sum) for a
    xarray dataArray, generating a new xarray dataArray
    
    INPUT -
    df_raw: 2D xarray DataArray
    stride: Size of the non-overlap square aggregation
    var_name: new DataArray variable name
    mode: square aggregation function
    
    OUTPU -
    A new 2D xarray DataArray with non-overlap square aggregation
    """
    nrow, ncol = df_raw.values.squeeze().shape
    new_nrow,new_ncol = int(nrow/stride), int(ncol/stride)
    data = df_raw.values.squeeze()[0:(new_nrow*stride),0:(new_ncol*stride)]
    new_data = square_helper(data, new_nrow, new_ncol, mode)
    new_x = np.mean(df_raw.coords['x'].values[0:(new_ncol*stride)].reshape(-1,stride),axis=1)
    new_y = np.mean(df_raw.coords['y'].values[0:(new_nrow*stride)].reshape(-1,stride),axis=1)
    # create new datarray
    df = xr.DataArray(
        data=new_data,
        dims=["y", "x"],
        coords=dict(
            x= new_x,
            y= new_y,
        ),
    )
    df.name = var_name
    if df_raw.rio.crs is not None:
        df = df.rio.write_crs(df_raw.rio.crs)
    return df

In [15]:
def grid_reproject(df,ref_data,resample_method = Resampling.nearest):
    """
    DESCRIPTION: This is a function to reproject a xarray dataArray to the coordinates
    of another refrence xarray dataArray.
    
    INPUT - 
    df: The xarray DataArray needing to be reprojected to the new grids
    ref_data: The reference xarray DataArray with reference coordinates (the new grids)
    resample_method: resample method
    
    OUTPUT -
    A new xarray DataArray reprojected to the coordinates of ref_data 
    """
    df = df.rio.reproject_match(ref_data, resampling = Resampling.nearest, nodata= np.nan)
    df = df.assign_coords({
        "x": ref_data.x,
        "y": ref_data.y,
    })
    return df

In [16]:
def points_in_grid(points_df,ref_data):
    """
    DESCRIPTION: This function finds how many points are in the grids centered 
    at the coordinates of the reference data
    
    INPUT -
    points_df: xarray DataArray that needs to be count 
    ref_data: xarray DataArray with reference coordinates
    
    OUTPUT -
    How many points are in each grid of the reference coordinates
    """
    long_list = ref_data.coords['x'].values
    lat_list = ref_data.coords['y'].values
    long_list = np.append(long_list, long_list[-1]+ref_data.rio.resolution()[0])
    lat_list = np.append(lat_list, lat_list[-1]+ref_data.rio.resolution()[1])
    count = np.zeros(ref_data.values.shape)
    grid_index = points_df.apply(lambda row: find_grid_index(row['Long'],row['Lat'],long_list,lat_list), axis=1)
    for x in grid_index:
        if x[0]!=-1 and x[1]!=-1:
            count[x[0],x[1]] += 1
    return count

In [17]:
def aggregate_count(count,block_size):
    """
    DESCRIPTION: This function apply square sum aggregation with sride 1 for 2D numpy array
    with same padding
    
    INPUT -
    count: 2D numpy array
    block_size: the size of the square sum aggregation
    
    OUTPUT - 
    a new 2D numpy array with square sum aggreation in the same size
    """
    if block_size%2 != 1:
        raise Exception('block_size must be an odd integer!')
    half = block_size//2
    new_count = np.zeros(count.shape)
    count = np.pad(count,half)
    for nrow in range(half,count.shape[0]-half):
        for ncol in range(half,count.shape[1]-half):
            new_count[nrow-half,ncol-half] = np.sum(count[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
    return new_count

In [18]:
def print_summary(data):
    """
    DESCRIPTION: This is a function to print the sumamry of a xarray DataArray or Dataset
    
    INPUT - 
    data: xarray DataArray or Dataset
    
    OUTPUT - 
    summary statistics of data
    """
    print(f"shape: {data.rio.shape}")
    print(f"resolution: {data.rio.resolution()}")
    print(f"coordinates boundary: {data.rio.bounds()}")
    print(f"CRS: {data.rio.crs}")

# Specify reference coordinates/grids

1. The spatial distribution of population in 2015 South Africa
2. zaf_ppp_2015_1km_Aggregated.tif
3. https://hub.worldpop.org/geodata/summary?id=33887

Here we use the grids of Uzbekistan population data as the reference data.
All other data will be reprojected to this reference grids to make all data consistent in space.

## Population -- coordinate reference

### Read in population data

In [19]:
pop_raw = rioxarray.open_rasterio(path_pop_count)
display(pop_raw)
print_summary(pop_raw)

<xarray.DataArray (band: 1, y: 2983, x: 2586)>
[7714038 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 16.46 16.47 16.48 16.49 ... 37.98 37.99 37.99 38.0
  * y            (y) float64 -22.13 -22.14 -22.15 ... -46.96 -46.97 -46.98
    spatial_ref  int64 0
Attributes:
    STATISTICS_MAXIMUM:  52822.44921875
    STATISTICS_MEAN:     33.824253836407
    STATISTICS_MINIMUM:  0
    STATISTICS_STDDEV:   339.48023586116
    _FillValue:          -99999.0
    scale_factor:        1.0
    add_offset:          0.0

shape: (2983, 2586)
resolution: (0.0083333333, -0.0083333333)
coordinates boundary: (16.457083269328677, -46.983749590994, 38.00708318312868, -22.125416357093997)
CRS: EPSG:4326


## Clean variables and dimensions

### Select band index at 0 

In [20]:
pop = pop_raw.isel(band=0)
pop

<xarray.DataArray (y: 2983, x: 2586)>
[7714038 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 16.46 16.47 16.48 16.49 ... 37.98 37.99 37.99 38.0
  * y            (y) float64 -22.13 -22.14 -22.15 ... -46.96 -46.97 -46.98
    spatial_ref  int64 0
Attributes:
    STATISTICS_MAXIMUM:  52822.44921875
    STATISTICS_MEAN:     33.824253836407
    STATISTICS_MINIMUM:  0
    STATISTICS_STDDEV:   339.48023586116
    _FillValue:          -99999.0
    scale_factor:        1.0
    add_offset:          0.0

### Drop extra coordinates dimensions beyond latitude and longtitude


In [21]:
# aggregate to larger grid scale
pop = xarray_square_aggregate(pop_raw,stride=10,
                             var_name="population", 
                             mode="sum")

In [22]:
# pop = pop.reset_coords(names=['band'],drop=True)
# pop.name = 'population'
pop

<xarray.DataArray 'population' (y: 298, x: 258)>
array([[-9999900., -9999900., -9999900., ..., -9999900., -9999900.,
        -9999900.],
       [-9999900., -9999900., -9999900., ..., -9999900., -9999900.,
        -9999900.],
       [-9999900., -9999900., -9999900., ..., -9999900., -9999900.,
        -9999900.],
       ...,
       [-9999900., -9999900., -9999900., ..., -9999900., -9999900.,
        -9999900.],
       [-9999900., -9999900., -9999900., ..., -3799962., -6499935.,
        -9999900.],
       [-9999900., -9999900., -9999900., ...,  -499995.,        0.,
        -7899921.]], dtype=float32)
Coordinates:
  * x            (x) float64 16.5 16.58 16.67 16.75 ... 37.67 37.75 37.83 37.92
  * y            (y) float64 -22.17 -22.25 -22.33 ... -46.75 -46.83 -46.92
    spatial_ref  int64 0

### Make population DataArray as the reference grids

In [23]:
ref_da = pop.copy()
ref_ds = ref_da.to_dataset()
ref_da

<xarray.DataArray 'population' (y: 298, x: 258)>
array([[-9999900., -9999900., -9999900., ..., -9999900., -9999900.,
        -9999900.],
       [-9999900., -9999900., -9999900., ..., -9999900., -9999900.,
        -9999900.],
       [-9999900., -9999900., -9999900., ..., -9999900., -9999900.,
        -9999900.],
       ...,
       [-9999900., -9999900., -9999900., ..., -9999900., -9999900.,
        -9999900.],
       [-9999900., -9999900., -9999900., ..., -3799962., -6499935.,
        -9999900.],
       [-9999900., -9999900., -9999900., ...,  -499995.,        0.,
        -7899921.]], dtype=float32)
Coordinates:
  * x            (x) float64 16.5 16.58 16.67 16.75 ... 37.67 37.75 37.83 37.92
  * y            (y) float64 -22.17 -22.25 -22.33 ... -46.75 -46.83 -46.92
    spatial_ref  int64 0

# Data Preprocessing

Process age and gender population data

1. Albania 100m Age and Gender structures in 2020
2.
3. https://www.worldpop.org/geodata/summary?id=50113

We have one Geotiff file for each sex class and each age bin, here we aggregate the 36 Geotiff files into one clean csv file, which will be used to calculate death data at each grid later.

### Set sex bins and age bins 

In [24]:
# sex_bins = ['Female','Male']
# age_bins = ['Age <1 year', 'Age 1 to 4','Age 5 to 9',
#             'Age 10 to 14','Age 15 to 19','Age 20 to 24',
#             'Age 25 to 29','Age 30 to 34','Age 35 to 39',
#             'Age 40 to 44','Age 45 to 49','Age 50 to 54',
#             'Age 55 to 59','Age 60 to 64','Age 65 to 69',
#             'Age 70 to 74','Age 75 to 79','Age 80 plus']
# age_start_bins = [0,1] + list(np.arange(5,80+1,5))

In [25]:
# %%time
# age_sex_df = None
# for sex in sex_bins:
#     for file_no, age in enumerate(age_bins):
#         file_name ="alb_"+sex[0].lower()+"_"+str(age_start_bins[file_no])+"_2020_constrained.tif"
#         col_name = sex + ' ' + age
#         age_sex_raw = xr.open_dataarray(os.path.join(dir_pop_agesex,file_name))
#         age_sex = xarray_square_aggregate(age_sex_raw,stride=10,var_name=col_name, mode="sum")
#         age_sex = grid_reproject(age_sex, pop)
#         df = age_sex.to_dataframe()
#         df = df.drop("spatial_ref",axis=1)
#         age_sex_df = df if age_sex_df is None else age_sex_df.merge(df, left_index=True, right_index=True) 
#         print(col_name)

### Rename and take a look at age and sex distribution

In [26]:
# %%time
# age_sex_df = age_sex_df.reset_index().rename({'x':'Long','y':'Lat'},
#                                              axis=1)
# age_sex_df

In [27]:
# %%time
# age_sex_df.to_csv(path_agesex)

## Process GDP data

1. Gridded global datasets for Gross Domestic Product and Human Development Index over 1990-2015
2. GDP_PPP_30arcsec_v3.nc
3. https://datadryad.org/stash/dataset/doi:10.5061/dryad.dk1j0

### Read in raw gdp data 

In [28]:
gdp_raw = rioxarray.open_rasterio(path_gdp)
display(gdp_raw)
print_summary(gdp_raw)

<xarray.DataArray 'GDP_PPP' (time: 3, y: 21600, x: 43200)>
[2799360000 values with dtype=float32]
Coordinates:
  * time         (time) float64 1.99e+03 2e+03 2.015e+03
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 90.0 89.99 89.98 89.97 ... -89.98 -89.99 -90.0
    spatial_ref  int64 0
Attributes:
    long_name:      Gross Domestic Production (GDP) (PPP)
    missing_value:  -9
    units:          ('constant 2011 international US dollar', 'constant 2011 ...
    _FillValue:     -9.0
    scale_factor:   1.0
    add_offset:     0.0

shape: (21600, 43200)
resolution: (0.008333333380429452, -0.00833333342752775)
coordinates boundary: (-180.00000101727616, -90.00000101729967, 180.00000101727616, 90.0000010172997)
CRS: None


### Clean gdp variables and dimensions

In [29]:
# select year 2015 gdp 
gdp = gdp_raw.isel(time=2)
# drop extra coordinates dimensions beyond latitude and longtitude
gdp = gdp.reset_coords(names=['time'],drop=True)
# specify CRS
gdp = gdp.rio.write_crs("epsg:4326", inplace=True)

gdp

<xarray.DataArray 'GDP_PPP' (y: 21600, x: 43200)>
[933120000 values with dtype=float32]
Coordinates:
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 90.0 89.99 89.98 89.97 ... -89.98 -89.99 -90.0
    spatial_ref  int64 0
Attributes:
    long_name:      Gross Domestic Production (GDP) (PPP)
    missing_value:  -9
    units:          ('constant 2011 international US dollar', 'constant 2011 ...
    _FillValue:     -9.0
    scale_factor:   1.0
    add_offset:     0.0

### Reproject gdp to the refrence coordinates

In [30]:
# aggregate GDP
gdp_mean = xarray_square_aggregate(gdp,stride=10,
                             var_name="GDP_PPP_mean", 
                             mode="mean")

gdp_sum = xarray_square_aggregate(gdp,stride=10,
                             var_name="GDP_PPP_sum", 
                             mode="sum")

In [31]:
gdp_mean = grid_reproject(gdp_mean, ref_da)
gdp_sum = grid_reproject(gdp_sum, ref_da)

## Days with heat index > 35℃

In [32]:
heat_raw = rioxarray.open_rasterio(path_heatindex)[0]
display(heat_raw)
# 0: mean; 1: longitude boundary; 2: latitude boundary
print_summary(heat_raw)


/Users/yangshining/opt/miniconda3/lib/python3.9/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/Users/yangshining/opt/miniconda3/lib/python3.9/site-packages/rioxarray/_io.py:851: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  warnings.warn(str(rio_warning.message), type(rio_warning.message))  # type: ignore
/Users/yangshining/opt/miniconda3/lib/python3.9/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/Users/yangshining/opt/miniconda3/lib/python3.9/site-packages/rioxarray/_io.py:851: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  warnings.warn(str(rio_war

<xarray.Dataset>
Dimensions:                   (time: 1, x: 361, y: 181)
Coordinates:
  * time                      (time) object 2020-01-01 12:00:00
  * x                         (x) float64 -180.0 -179.0 -178.0 ... 179.0 180.0
  * y                         (y) float64 90.0 89.0 88.0 ... -88.0 -89.0 -90.0
    spatial_ref               int64 0
Data variables:
    anomaly-hi35-annual-mean  (time, y, x) float32 ...
Attributes: (12/30)
    activity_id:           ScenarioMIP
    climate_data_id:       CMIP6
    cmip6-ng:              Some daily data has been acquired through the CMIP...
    cmor_version:          3.4.0
    Conventions:           CF-1.7 CMIP-6.2
    creation_date:         2021-11-05T01:00:00Z
    ...                    ...
    parent_activity_id:    CMIP and ScenarioMIP
    parent_experiment_id:  piControl
    parent_mip_era:        CMIP6
    parent_time_units:     days since 1850-01-01
    realm:                 atmos
    table_id:              day

shape: (181, 361)
resolution: (1.0, -1.0)
coordinates boundary: (-180.5, -90.5, 180.5, 90.5)
CRS: None


In [33]:
# rename variable
heat_raw["days_heat_index"] = heat_raw["anomaly-hi35-annual-mean"]
heat_raw = heat_raw.drop(["anomaly-hi35-annual-mean"])

In [34]:
heat = heat_raw.isel(time=0)
heat = heat.reset_coords(names=['time'],drop=True)
heat = heat.rio.write_crs("epsg:4326", inplace=True)
heat = grid_reproject(heat, ref_da)
print_summary(heat)

shape: (298, 258)
resolution: (0.08333333300000001, -0.08333333300000001)
coordinates boundary: (16.457083269328677, -46.958749591094, 37.95708318332868, -22.125416357093997)
CRS: EPSG:4326


## Process Night life lights data

1. VIIRS night-time lights (2012-2016), South Africa
2. zaf_viirs_100m_2015.tiff
3. https://hub.worldpop.org/geodata/summary?id=18738 

### Read in raw nightlight data 

In [35]:
nl_raw = rioxarray.open_rasterio(path_nl)
display(nl_raw)
print_summary(nl_raw)

<xarray.DataArray (band: 1, y: 29827, x: 25856)>
[771206912 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 16.46 16.46 16.46 16.46 ... 38.0 38.0 38.0 38.0
  * y            (y) float64 -22.13 -22.13 -22.13 ... -46.98 -46.98 -46.98
    spatial_ref  int64 0
Attributes:
    _FillValue:    3.4028234663852886e+38
    scale_factor:  1.0
    add_offset:    0.0

shape: (29827, 25856)
resolution: (0.00083333333, -0.00083333333)
coordinates boundary: (16.457916615829987, -46.98291627736, 38.00458319630999, -22.12708304345)
CRS: EPSG:4326


### Clean nightlight variables and dimensions

In [36]:
# Aggregate the 100m nightlight to 1km with square mean
nl = xarray_square_aggregate(nl_raw,stride=100,
                             var_name="nightlight", 
                             mode="mean")

/Users/yangshining/opt/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


### Reproject the nightlight to reference coordinates

In [37]:
nl = grid_reproject(nl, ref_da)
nl.name = 'nightlight'
display(nl)

<xarray.DataArray 'nightlight' (y: 298, x: 258)>
array([[       inf,        inf,        inf, ...,        inf,        inf,
               inf],
       [       inf,        inf,        inf, ...,        inf,        inf,
               inf],
       [       inf,        inf,        inf, ...,        inf,        inf,
               inf],
       ...,
       [       inf,        inf,        inf, ...,        inf,        inf,
               inf],
       [       inf,        inf,        inf, ...,        inf,        inf,
               inf],
       [       inf,        inf,        inf, ..., 0.07267279,        inf,
               inf]], dtype=float32)
Coordinates:
  * x            (x) float64 16.5 16.58 16.67 16.75 ... 37.67 37.75 37.83 37.92
  * y            (y) float64 -22.17 -22.25 -22.33 ... -46.75 -46.83 -46.92
    spatial_ref  int64 0
Attributes:
    _FillValue:  nan

## Merge GDP, population, nightlight into one dataset

In [38]:
merge_ds = xr.merge([gdp_mean, gdp_sum, pop, nl, heat])
display(merge_ds)
print_summary(merge_ds) 

<xarray.Dataset>
Dimensions:          (x: 258, y: 298)
Coordinates:
  * x                (x) float64 16.5 16.58 16.67 16.75 ... 37.75 37.83 37.92
  * y                (y) float64 -22.17 -22.25 -22.33 ... -46.75 -46.83 -46.92
    spatial_ref      int64 0
Data variables:
    GDP_PPP_mean     (y, x) float32 1.098e+03 3.427e+03 1.112e+03 ... -9.0 -9.0
    GDP_PPP_sum      (y, x) float32 1.098e+05 3.427e+05 ... -900.0 -900.0
    population       (y, x) float32 -1e+07 -1e+07 -1e+07 ... -5e+05 0.0 -7.9e+06
    nightlight       (y, x) float32 inf inf inf inf inf ... inf 0.07267 inf inf
    days_heat_index  (y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    _FillValue:  nan

shape: (298, 258)
resolution: (0.08333333300000001, -0.08333333300000001)
coordinates boundary: (16.457083269328677, -46.958749591094, 37.95708318332868, -22.125416357093997)
CRS: EPSG:4326


In [39]:
merge_df = merge_ds.to_dataframe().reset_index()
merge_df

,x,y,spatial_ref,GDP_PPP_mean,GDP_PPP_sum,population,nightlight,days_heat_index
0,16.498750,-22.167083,0,1097.975952,1.097976e+05,-9999900.0,inf,0.0
1,16.498750,-22.250416,0,-9.000000,-9.000000e+02,-9999900.0,inf,0.0
2,16.498750,-22.333750,0,-9.000000,-9.000000e+02,-9999900.0,inf,0.0
3,16.498750,-22.417083,0,12825.700195,1.282570e+06,-9999900.0,inf,0.0
4,16.498750,-22.500416,0,11214.549805,1.121455e+06,-9999900.0,inf,0.0
...,...,...,...,...,...,...,...,...
76879,37.915417,-46.583750,0,-9.000000,-9.000000e+02,-7199928.0,inf,0.0
76880,37.915417,-46.667083,0,-9.000000,-9.000000e+02,-5999940.0,inf,0.0
76881,37.915417,-46.750416,0,-9.000000,-9.000000e+02,-9999900.0,inf,0.0
76882,37.915417,-46.833750,0,-9.000000,-9.000000e+02,-9999900.0,inf,0.0


In [40]:
merge_df[merge_df['population']<=0]['population'].value_counts()

-9999900.000    60094
-4999949.500       18
 0.000             15
-4999950.000        8
-9899901.000        8
                ...  
-2099852.500        1
-3599308.750        1
-8899890.000        1
-1199937.875        1
-3799962.000        1
Name: population, Length: 1046, dtype: int64

In [41]:
merge_df[merge_df['GDP_PPP_mean']<=0]['GDP_PPP_mean'].value_counts()

-9.000000    60881
-2.393581        1
-3.332162        1
-7.015270        1
-4.380124        1
             ...  
-6.134604        1
-8.489740        1
-5.366002        1
-7.516031        1
-5.707229        1
Name: GDP_PPP_mean, Length: 113, dtype: int64

In [42]:
merge_df.isnull().sum()

x                  0
y                  0
spatial_ref        0
GDP_PPP_mean       0
GDP_PPP_sum        0
population         0
nightlight         0
days_heat_index    0
dtype: int64

In [43]:
# drop observations with missing population or missing GDP [note: missing was coded as -99999]
merge_df2 = (
    merge_df[(merge_df["population"] > 0) & (merge_df["GDP_PPP_mean"] > 0)]
    .copy()
    .reset_index(drop=True)
)
merge_df2

,x,y,spatial_ref,GDP_PPP_mean,GDP_PPP_sum,population,nightlight,days_heat_index
0,16.665417,-28.750416,0,2.546182e+01,2.546182e+03,0.471344,0.044446,0.0
1,16.665417,-28.833750,0,1.388521e+02,1.388521e+04,0.990747,0.055383,0.0
2,16.748750,-28.833750,0,1.121833e+01,1.121833e+03,0.146418,0.039771,0.0
3,16.748750,-28.917083,0,1.649256e+01,1.649256e+03,0.509858,0.041680,0.0
4,16.832083,-28.417083,0,1.193500e+05,1.193500e+07,1.583840,0.114101,0.0
...,...,...,...,...,...,...,...,...
11824,32.748750,-26.917083,0,1.318062e+06,1.318062e+08,9144.445312,0.152054,4.7
11825,32.748750,-27.000416,0,8.538637e+05,8.538637e+07,24504.955078,0.933610,4.7
11826,32.748750,-27.083750,0,1.550694e+05,1.550694e+07,1563.073486,0.042506,4.7
11827,32.748750,-27.167083,0,4.388440e+04,4.388440e+06,2362.568848,0.070886,4.7


In [44]:

# generate GDP PC
merge_df2["pc"] = merge_df2["GDP_PPP_sum"] / merge_df2["population"]

# rename x and y to longitude and latitude
merge_df2.rename({"x": "Long", "y": "Lat"}, axis="columns", inplace=True)

# drop spatial ref and band
merge_df2.drop(["spatial_ref"], axis="columns", inplace=True)

# take a look at the data
merge_df2


,Long,Lat,GDP_PPP_mean,GDP_PPP_sum,population,nightlight,days_heat_index,pc
0,16.665417,-28.750416,2.546182e+01,2.546182e+03,0.471344,0.044446,0.0,5.401958e+03
1,16.665417,-28.833750,1.388521e+02,1.388521e+04,0.990747,0.055383,0.0,1.401489e+04
2,16.748750,-28.833750,1.121833e+01,1.121833e+03,0.146418,0.039771,0.0,7.661835e+03
3,16.748750,-28.917083,1.649256e+01,1.649256e+03,0.509858,0.041680,0.0,3.234737e+03
4,16.832083,-28.417083,1.193500e+05,1.193500e+07,1.583840,0.114101,0.0,7.535480e+06
...,...,...,...,...,...,...,...,...
11824,32.748750,-26.917083,1.318062e+06,1.318062e+08,9144.445312,0.152054,4.7,1.441380e+04
11825,32.748750,-27.000416,8.538637e+05,8.538637e+07,24504.955078,0.933610,4.7,3.484453e+03
11826,32.748750,-27.083750,1.550694e+05,1.550694e+07,1563.073486,0.042506,4.7,9.920800e+03
11827,32.748750,-27.167083,4.388440e+04,4.388440e+06,2362.568848,0.070886,4.7,1.857487e+03


In [46]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
merge_df2.describe()

,Long,Lat,GDP_PPP_mean,GDP_PPP_sum,population,nightlight,days_heat_index,pc
count,11829.00000,11829.00000,11829.00000,11829.00000,11829.00000,11829.00000,11829.00000,11829.00000
mean,26.08855,-28.82759,546755.25000,54675524.00000,4238.38818,inf,0.10312,255133.07812
std,3.59091,3.05616,3014629.00000,301462912.00000,22694.38477,NaN,0.52601,12453571.00000
min,16.66542,-34.75042,0.32325,32.32452,0.02188,-0.00131,0.00000,0.23958
25%,23.66542,-31.41708,2491.11426,249111.43750,27.83011,0.02673,0.00000,3983.78638
50%,26.49875,-28.66708,17435.30273,1743530.25000,142.27455,0.05464,0.00000,11098.49219
75%,29.08208,-26.50042,195037.15625,19503716.00000,1557.27869,0.16994,0.00000,27890.83594
max,32.83208,-22.25042,89113664.00000,8911366144.00000,687290.62500,inf,5.30000,1345727232.00000


### Get percentiles for mapping

GDP percentiles - bottom 40%, 40-80%, 80-95%, above 95%

In [43]:
gdp_pc_values = merge_df2['GDP_PPP']
gdp_pc_cutoff = np.nanquantile(gdp_pc_values,[0.4, 0.8, 0.95, 1])

# turn off scientific notation in numpy arrays
np.set_printoptions(formatter={'float': '{:0.4f}'.format})
print(gdp_pc_cutoff)

[8577.7766 347377.3250 1855171.7500 89113664.0000]


Population percentiles - bottom 10%, 10-50%, 50-75%, 75-90%, 90-95%, 95-99%, above 99%

In [45]:
pop_values = merge_df2['population']
pop_cutoff = np.nanquantile(pop_values,[0.1, 0.5, 0.75, 0.9, 0.95, 0.99, 1])
pop_cutoff

array([5.6327, 142.2746, 1557.2787, 7305.6492, 14891.4525, 78722.1900,
       687290.6250])

# Data Analysis

## GDP rating

In [45]:
# # assign gdp rating 
# gdp_values = merge_df2['GDP_PPP']
# gdp_cutoff = np.nanquantile(gdp_values,[0.7,0.95])
# gdp_levels = np.array(['nan']*len(gdp_values))
# gdp_levels[~np.isnan(gdp_values)] = 'C'
# gdp_levels[gdp_values > gdp_cutoff[0]] = 'B'
# gdp_levels[gdp_values > gdp_cutoff[1]] = 'A'
# merge_df2['gdp_rating'] = gdp_levels
# merge_df2['gdp_rating'].value_counts()

In [46]:
# %%time
# merge_df2 = merge_df2.rename({'x':'Long','y':'Lat'},axis=1)
# if read in age sex csv, there will be some decimal rounding difference due to float type
# merge_df2['Lat_5_decimal'] = np.round(merge_df2['Lat'],5).astype('str')
# merge_df2['Long_5_decimal'] = np.round(merge_df2['Long'],5).astype('str')
# display(merge_df2.head())


# Save the final data

In [47]:
%%time
merge_df2.to_csv(os.path.join(path_data_out, "zaf_final_data_10km_heatindex_0719SY.csv"))

CPU times: user 93.6 ms, sys: 24 ms, total: 118 ms
Wall time: 146 ms


# Total runtime

In [47]:
end_time = time.perf_counter ()
print(round((end_time - start_time)/60,2), "minutes")

9.21 minutes
